## TripletLoss

As first introduced in the FaceNet paper, TripletLoss is a loss function that trains a neural network to closely embed features of the same class while maximizing the distance between embeddings of different classes.  To do this an anchor  is chosen along with one negative and one positive sample.
![fig3](https://user-images.githubusercontent.com/18154355/61485418-1cbb1f00-a96f-11e9-8de8-3c46eef5a7dc.png)

**The loss function is described as a Euclidean distance function:**

![function](https://user-images.githubusercontent.com/18154355/61484709-7589b800-a96d-11e9-9c3c-e880514af4b7.png)

Where A is our anchor input,  P is the positive sample input,  N is the negative sample input, and alpha is some margin you use to specify when a triplet has become too "easy" and you no longer want to adjust the weights from it.

## Setup

In [1]:
!pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.2 MB/s eta 0:00:00


In [2]:
import io
import numpy as np

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

## Prepare the Data

In [4]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)

train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
print(train_dataset)

<MapDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


## Build the Model

![fig2](https://user-images.githubusercontent.com/18154355/61485417-1cbb1f00-a96f-11e9-8d6a-94964ce8c4db.png)

In [6]:
#defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

## Train and Evaluate

In [8]:
optimizer = tf.keras.optimizers.Adam(0.001)
loss = tfa.losses.TripletSemiHardLoss()
# Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,metrics = ['accuracy'])


In [9]:
# Train the network
history = model.fit(
    train_dataset,
    epochs=100)

Epoch 1/100
1875/1875 [==============================] - 21s 7ms/step - loss: 0.5926 - accuracy: 0.0017
Epoch 2/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4571 - accuracy: 6.0000e-04
Epoch 3/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4219 - accuracy: 6.0000e-04
Epoch 4/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4021 - accuracy: 6.8333e-04
Epoch 5/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3915 - accuracy: 8.6667e-04
Epoch 6/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3823 - accuracy: 7.5000e-04
Epoch 7/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3782 - accuracy: 0.0010
Epoch 8/100
1875/1875 [==============================] - 11s 6ms/step - loss: 0.3717 - accuracy: 9.8333e-04
Epoch 9/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3672 - accuracy: 8.8333e-04
Epoch 10/100
1875/1875 [=============

In [10]:
# Evaluate the network
results = model.predict(test_dataset)

313/313 [==============================] - 1s 3ms/step


In [11]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Embedding Projector

The vector and metadata files can be loaded and visualized here: https://projector.tensorflow.org/

You can see the results of our embedded test data when visualized with UMAP:
![embedding](https://user-images.githubusercontent.com/18154355/61600295-e6470380-abfd-11e9-8a00-2b25e7e6916f.png)
